In [55]:
import numpy as np

# 初始化参数和矩阵
def initialize_matrices(n, m, Q_val=1):
    P_i_j = np.eye(n)       # 初始矩阵 P_i^j
    Q = Q_val * np.eye(n)   # 矩阵 Q
    A = np.array([[0, 0, 0, 0], 
                  [-2, -5, -10, 0], 
                  [1, 2, -2, 1], 
                  [1, 0, 0, -2]])  # 系统矩阵 A
    B = np.array([[1, 0],
                  [-1, 0], 
                  [-1, 2], 
                  [1, 1]])  # 系统矩阵 B
    K_i_j = np.zeros((m, n))  # 初始反馈增益矩阵 K 为零矩阵
    
    return P_i_j, Q, A, B, K_i_j



In [56]:
# 定义计算 Hj 和 Kj 的辅助函数
def compute_H_K(P_i_j, A, B):
    H_i_j = A.T @ P_i_j + P_i_j @ A
    K_i_j = -B.T @ P_i_j
    return H_i_j, K_i_j

# 定义更新 P_i_j 的函数
def update_P(P_i_j, H_i_j, Q, K_i_j, K_i_j_prev, delta_j):
    return P_i_j + delta_j * (H_i_j + Q - (K_i_j.T @ K_i_j))





In [57]:
# 主求解循环
def solve_K(n, m, epsilon=1, max_iters=10000):
    # 初始化各种参数
    P_i_j, Q, A, B, K_i_j = initialize_matrices(n, m, Q_val=1)
    K_i_j_prev = K_i_j
    P_i_j_0 = P_i_j
    j = 0
    k = 0
    
    while j < max_iters:
        delta_j = 1 / (j + 1)  # 动态学习率
        
        # 计算 Hj 和 Kj
        H_i_j, K_i_j = compute_H_K(P_i_j, A, B)

        # 更新 P_i_j
        P_i_j_next = update_P(P_i_j, H_i_j, Q, K_i_j, K_i_j_prev, delta_j)

        # 限制 P 矩阵的范数
        if np.linalg.norm(P_i_j_next, ord='fro') > 10 * (k + 1):
            # P_i_j_next = (10 * (j + 1)) * P_i_j_next / np.linalg.norm(P_i_j_next, ord='fro')
            P_i_j = P_i_j_0
            k = k + 1
        # 判断收敛条件
        elif np.linalg.norm(P_i_j_next - P_i_j, ord='fro') < epsilon:
            print(f"Converged at iteration {j}")
            return K_i_j
        
        # 否则继续迭代
        else :
            P_i_j = P_i_j_next
        K_i_j_prev = K_i_j
        j += 1
    
    print("Maximum iterations reached without convergence.")
    return K_i_j



In [58]:
# 使用
n = 4  # 系统状态维度
m = 2  # 控制输入维度
K_matrix = solve_K(n, m)
print("Resulting K matrix:", K_matrix)

Converged at iteration 14
Resulting K matrix: [[-0.88292477 -0.51986475  0.69146238 -0.50845309]
 [-0.52662722  0.77007608 -2.47633136 -0.30515638]]
